In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import scipy.stats as stats
import pandas as pd
from tqdm import tqdm
from statsmodels.stats.multitest import fdrcorrection
import networkx as nx
import statsmodels.formula.api as smf

In [ ]:
def plot_prob_points(*args, facecolors=None, alpha=None, capsize=10, elinewidth=10, capthick=6, dot_size=400, yticks=None, ytickslabels=None, ymin=None, ymax=None, random_state=10, figsize=(10,10)):
    np.random.seed(random_state)
    SPREAD_LENGTH = 0.1
    
    x_list = np.arange(1.0, 1.0+0.5*len(args), 0.5)
    y_list = list(map(lambda x: x.mean(), args))
    error_list = list(map(lambda x: x.std()/np.sqrt(len(x)), args))
    spread_list = x_list - SPREAD_LENGTH/2.0
    
    # create points for means and standard errors
    fig, ax = plt.subplots(1,1, figsize=figsize)
    ax.errorbar(x_list, y_list, error_list, ls="none", capsize=capsize, elinewidth=elinewidth, capthick=capthick, ecolor="black")
    ax.scatter(x_list, y_list, s=dot_size, c="black", zorder=10)
    
    if facecolors is None:
        facecolors = np.repeat("#cccccc", len(args))
    if alpha is None:
        alpha = np.repeat(0.5, len(args))

    if type(facecolors) == str:
        facecolors = np.repeat(facecolors, len(args))
    if type(alpha) == float or type(alpha) == int:
        alpha = np.repeat(alpha, len(args))
    
    # draw each point
    for i, arg in enumerate(args):
        ax.scatter(
            np.random.rand(len(arg))*SPREAD_LENGTH + spread_list[i], 
            arg, 
            c=facecolors[i], 
            edgecolors='none', 
            s=dot_size, 
            alpha=alpha[i],
            zorder=0
        )

    ax.set_xlim([1.0-0.4, max(x_list)+0.4])
    ax.set_xticks(x_list, np.repeat("", len(args)))
    ax.spines[['right', 'top']].set_visible(False)
    plt.setp(ax.spines.values(), linewidth=8)
    
    if ymin and ymax:
        ax.set_ylim([ymin, ymax])
    
    if yticks:
        if ytickslabels:
            ax.set_yticks(yticks, ytickslabels)
        else:
            ax.set_yticks(yticks)


def plot_histogram(x_boostrap, x):
    fig, ax = plt.subplots(1, figsize=(10,10))
    N, bins, patches = ax.hist(x_boostrap, bins=int(np.sqrt(50_000)), density=True, fc=(0,0,1,0.5))
    ax.plot([x, x], [0, np.max(N)*0.5], c="r", linewidth=10)
    ax.set_ylim([0, np.max(N)*1.1])

    ax.set_xticks([0], [""])
    ax.set_yticks([0], [""])
    ax.spines[['right', 'top', 'left']].set_visible(False)
    plt.setp(ax.spines.values(), linewidth=8)

In [ ]:
MAIN_DIR = "/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn"
connectivity = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_2.mat")["C_net_2"]

#connectivity is subject x condition x network x network
diff_transicion = connectivity[:, 1, :, :]-connectivity[:, 0, :, :]
diff_alteracion = connectivity[:, 2, :, :]-connectivity[:, 0, :, :]
diff_recuperacion = connectivity[:, 3, :, :]-connectivity[:, 0, :, :]

n_sessions = 20

network_names = ["DMN", "VIS", "FP", "Rew", "DA", "VA", "SN", "CO", "SMD", "SML", "AUD", "PM", "MTL", "un"]

REMOVE_REPOSO = 7-1
REMOVE_ALTERACION = 5-1

#### Calculo de conectividad intra e inter

In [ ]:
connectivity_regions = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_300.mat")["C"]
connectivity_regions_controles = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300_controles/connectivity_300.mat")["C"]

intra_inter_df = pd.DataFrame([], columns=["intra", "inter", "session", "cond", "network"])
intra_inter_df_controles = pd.DataFrame([], columns=["intra", "inter", "session", "cond", "network"])

func_network_affiliation = np.loadtxt(f"{MAIN_DIR}/scripts/func_networks/masks/ROIs_300inVol_MNI/ROIs_CommunityAffiliation.txt").astype(int)
func_network_unique_list = np.unique(func_network_affiliation)

inter_per_network = np.zeros((20,4,13))
intra_per_network = np.zeros((20,4,13))
for iSess in tqdm(range(n_sessions)):
    for iCond in range(4):
        for index, iFunc in enumerate(func_network_unique_list):
            if iFunc == 18:
                continue
            func_network = func_network_affiliation == iFunc
            
            inter_trans = connectivity_regions[iSess, iCond, :, :][func_network][:,func_network == False]
            intra_trans = connectivity_regions[iSess, iCond, :, :][func_network][:,func_network]

            inter_per_network[iSess, iCond, index] = np.mean(connectivity_regions[iSess, iCond, :, :][func_network][:,func_network == False])
            intra_per_network[iSess, iCond, index] = np.mean(connectivity_regions[iSess, iCond, :, :][func_network][:,func_network])
            
            intra_inter_df.loc[len(intra_inter_df.index)] = [np.mean(intra_trans), np.mean(inter_trans), iSess+1, iCond+1, iFunc] 

inter_per_network_controles = np.zeros((10,4,13))
intra_per_network_controles = np.zeros((10,4,13))
for iSess in tqdm(range(10)):
    for iCond in range(4):
        for index, iFunc in enumerate(func_network_unique_list):
            if iFunc == 18:
                continue
            func_network = func_network_affiliation == iFunc

            inter_trans_controles = connectivity_regions_controles[iSess, iCond, :, :][func_network][:,func_network == False]
            intra_trans_controles = connectivity_regions_controles[iSess, iCond, :, :][func_network][:,func_network]
            
            inter_per_network_controles[iSess, iCond, index] = np.mean(connectivity_regions_controles[iSess, iCond, :, :][func_network][:,func_network == False])
            intra_per_network_controles[iSess, iCond, index] = np.mean(connectivity_regions_controles[iSess, iCond, :, :][func_network][:,func_network])

            intra_inter_df_controles.loc[len(intra_inter_df_controles.index)] = [np.mean(intra_trans_controles), np.mean(inter_trans_controles), iSess+1, iCond+1, iFunc] 

intra_inter_df["session"] = intra_inter_df["session"].astype(int)
intra_inter_df["cond"] = intra_inter_df["cond"].astype(int)
intra_inter_df["network"] = intra_inter_df["network"].astype(int)
intra_inter_df_controles["session"] = intra_inter_df_controles["session"].astype(int)
intra_inter_df_controles["cond"] = intra_inter_df_controles["cond"].astype(int)
intra_inter_df_controles["network"] = intra_inter_df_controles["network"].astype(int)

#intra_inter_df.to_csv("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/statistics/tables_for_model/intra_inter_network.csv")

intra_sessions = np.zeros((n_sessions, 4)) # sesiones x condiciones
inter_sessions = np.zeros((n_sessions, 4)) # sesiones x condiciones
intra_sessions_controles = np.zeros((10, 4)) # sesiones x condiciones
inter_sessions_controles = np.zeros((10, 4)) # sesiones x condiciones
for iCond in range(4):
    df_intra = intra_inter_df[intra_inter_df["cond"] == iCond+1]
    df_intra_controles = intra_inter_df_controles[intra_inter_df_controles["cond"] == iCond+1]
    
    for iSess in range(n_sessions):
        df = df_intra[df_intra["session"] == iSess+1]

        intra_sessions[iSess, iCond] = df["intra"].mean()
        inter_sessions[iSess, iCond] = df["inter"].mean()

    for iSess in range(10):
        df_controles = df_intra_controles[df_intra_controles["session"] == iSess+1]

        intra_sessions_controles[iSess, iCond] = df_controles["intra"].mean()
        inter_sessions_controles[iSess, iCond] = df_controles["inter"].mean()

intra_sessions_controles[REMOVE_REPOSO, 0] = np.nan
inter_sessions_controles[REMOVE_ALTERACION, 0] = np.nan

inter_per_network_3d = inter_per_network.copy()
inter_per_network = np.mean(inter_per_network, axis=0)
intra_per_network = np.mean(intra_per_network, axis=0)
inter_per_network_controles = np.mean(inter_per_network_controles, axis=0)
intra_per_network_controles = np.mean(intra_per_network_controles, axis=0)

#### Inter normalizada modelo mixto

In [ ]:
def calc_internorm_stats():
    reposo_mean_inter = inter_sessions[:, 0].mean()
    reposo_mean_intra = intra_sessions[:, 0].mean()

    df_data = {"session": [], "cond": [], "internorm": [], "intranorm": []}
    for iSess in range(20):
        for iCond in range(4):
            df_data["session"].append(iSess)
            df_data["cond"].append(iCond)
            df_data["internorm"].append(inter_sessions[iSess, iCond]/reposo_mean_inter)
            df_data["intranorm"].append(intra_sessions[iSess, iCond]/reposo_mean_intra)

    df = pd.DataFrame(df_data)
    df['cond'] = df['cond'].astype('category')
    df['session'] = df['session'].astype('category')

    md = smf.mixedlm("internorm ~ cond", df, groups=df["session"])
    mdf = md.fit()

    summary = mdf.summary().__str__()
    with open("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-global/estadistica/internorm.txt", "w") as file:
        file.write(summary)

    all_data = pd.DataFrame({"pvalues": mdf.pvalues[:-1], "tvalues": mdf.tvalues[:-1]})
    significant_data = all_data[all_data["pvalues"] < 0.05]

    return all_data, significant_data, df

all_data_internorm, significant_data_internorm, df_data = calc_internorm_stats()
all_data_internorm


In [ ]:
p_values = np.array(all_data_internorm["pvalues"].tolist())
h, pcorregido = fdrcorrection(p_values[1:], alpha=0.05, method="indep", is_sorted=False)

h, pcorregido

#### Bootstrap para conectividad por pares de redes

In [ ]:
connectivity_controles = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300_controles/connectivity_2.mat")["C_net_2"]

repo_list = connectivity_controles[:,0,:,:]
trans_list = connectivity_controles[:,1,:,:]
alter_list = connectivity_controles[:,2,:,:]
recu_list = connectivity_controles[:,3,:,:]

repo_list[REMOVE_REPOSO, :, :] = np.nan
alter_list[REMOVE_ALTERACION, :, :] = np.nan

avp_conns = sio.loadmat(f"{MAIN_DIR}/results/connectivity/conn_list_network.mat")
avp_diff_transicion = avp_conns["trans_list"] - avp_conns["repo_list"]
avp_diff_alteracion = avp_conns["alter_list"] - avp_conns["repo_list"]
avp_diff_recuperacion = avp_conns["recu_list"] - avp_conns["repo_list"]
controles_diff_transicion = trans_list - repo_list
controles_diff_alteracion = alter_list - repo_list
controles_diff_recuperacion = recu_list - repo_list

signif_networks_trans = [
("DMN", "DA"),
("VIS", "Rew"),
("VIS", "DA"),
("VIS", "VA"),
("VIS", "SMD"),
("VIS", "AUD"),
("VIS", "MTL"),
("Rew", "DA"),
("DA", "PM"),
("DA", "MTL"),
("SN", "MTL"),
]

signif_networks_alter = [
("VIS", "VA"),
("VIS", "SMD"),
("VIS", "SML"),
("VIS", "AUD"),
("VIS", "PM"),
("Rew", "SN"),
("DA", "PM"),
("VA", "SMD"),
("VA", "PM"),
("SN", "AUD"),
("SN", "MTL"),
("SMD", "SML"),
("SMD", "AUD"),
("SML", "AUD")
]

signif_networks_recup = [
("DMN", "SN"),
("VIS", "VIS"),
("VIS", "FP"),
("VIS", "VA"),
("VIS", "SN"),
("VIS", "CO"),
("VIS", "SMD"),
("VIS", "SML"),
("VIS", "AUD"),
("VIS", "PM"),
("Rew", "SN"),
("DA", "PM"),
("VA", "SML"),
("SN", "AUD"),
("SMD", "SML"),
("SMD", "AUD"),
("SMD", "MTL"),
("AUD", "PM"),
("AUD", "MTL"),
]

np.random.seed(42)
def bootstrap_connectivity_per_network(condicion_seleccionada: str):

    controles_diff_list = {"trans": controles_diff_transicion, "alter": controles_diff_alteracion, "recup": controles_diff_recuperacion}
    avp_diff_list = {"trans": avp_diff_transicion, "alter": avp_diff_alteracion, "recup": avp_diff_recuperacion}

    controles_diff = controles_diff_list[condicion_seleccionada]
    avp_diff = avp_diff_list[condicion_seleccionada]

    df_cols = {"network1": [], "network2": [], "z": [], "p": [], "signif": []}
    N = 50_000
    histograms = {}
    for i in tqdm(range(13)):
        for j in range(i+1, 13):
            d_list = np.zeros(N)
            for perm in range(N):
                data = np.hstack((controles_diff[:,i,j], controles_diff[:,i,j], avp_diff[:,i,j]))
                np.random.shuffle(data)
                d = np.nanmean(data[0:20]) - np.nanmean(data[20:40])
                d_list[perm] = d

            x = np.nanmean(controles_diff[:,i,j]) - np.nanmean(avp_diff[:,i,j])
            
            if x > 0:
                p = np.sum(d_list > x)/len(d_list)
                z = -1
            else:
                p = np.sum(d_list < x)/len(d_list)
                z = 1

            histograms[f"{network_names[i]}_{network_names[j]}"] = (d_list, x)

            df_cols["network1"].append(network_names[i])
            df_cols["network2"].append(network_names[j])
            df_cols["z"].append(z)
            df_cols["p"].append(p)

    df = pd.DataFrame({"network1": df_cols["network1"], "network2": df_cols["network2"], "z": df_cols["z"], "p": df_cols["p"]})
    new_df = pd.DataFrame([], columns=["network1", "network2", "z", "p"]) #solo incluyo las conectividades que son significativas en AVP

    redes_significativas = {"trans": signif_networks_trans, "alter": signif_networks_alter, "recup": signif_networks_recup}

    for n1, n2 in redes_significativas[condicion_seleccionada]: 
        row = df[(df["network1"] == n1) & (df["network2"] == n2)]
        new_df = pd.concat([new_df, row])

    h, p_corrected = fdrcorrection(new_df["p"].values, alpha=0.05, method="indep", is_sorted=False)

    return new_df, new_df[h], histograms, p_corrected

In [ ]:
df_transicion, corrected_transicion, hist_transicion, p_corr_transicion = bootstrap_connectivity_per_network("trans")
df_alteracion, corrected_alteracion, hist_alteracion, p_corr_alteracion = bootstrap_connectivity_per_network("alter")
df_recuperacion, corrected_recuperacion, hist_recuperacion, p_corr_recuperacion = bootstrap_connectivity_per_network("recup")

In [ ]:
corrected_transicion

In [ ]:
corrected_alteracion

In [ ]:
corrected_recuperacion

In [ ]:
def bootstrap_variacion(condicion: str) -> (float, float):
    condiciones = {"trans": 1, "alter": 2, "recup": 3}
    index = condiciones[condicion]

    connectivity_regions[:,:,np.eye(300).astype(bool)] = np.nan
    connectivity_regions_controles[:,:,np.eye(300).astype(bool)] = np.nan
    connectivity_regions_controles[REMOVE_REPOSO, 0] = np.nan
    connectivity_regions_controles[REMOVE_ALTERACION, 2] = np.nan
    connectivity_regions_controles[REMOVE_REPOSO, 0] = np.nan
    connectivity_regions_controles[REMOVE_ALTERACION, 2] = np.nan
    variacion = np.zeros((20,4))
    desvio = np.zeros((20,4))
    variacion_controles = np.zeros((10,4))
    desvio_controles = np.zeros((10,4))
    for iSess in range(20):
        for iCond in range(4):
            variacion[iSess,iCond] = np.nanstd(connectivity_regions[iSess, iCond, 12:, 12:])/np.nanmean(connectivity_regions[iSess, iCond, 12:, 12:])
            desvio[iSess,iCond] = np.nanstd(connectivity_regions[iSess, iCond, 12:, 12:])

    for iSess in range(10):
        for iCond in range(4):
            variacion_controles[iSess,iCond] = np.nanstd(connectivity_regions_controles[iSess, iCond, 12:, 12:])/np.nanmean(connectivity_regions_controles[iSess, iCond, 12:, 12:])
            desvio_controles[iSess,iCond] = np.nanstd(connectivity_regions_controles[iSess, iCond, 12:, 12:])

    variacion_diff = variacion[:,index] - variacion[:,0]
    variacion_controles_diff = variacion_controles[:,index] - variacion_controles[:,0]
    variacion_controles_diff_condicion = variacion_controles_diff[~np.isnan(variacion_controles_diff)]

    N_perm = 50_000
    d_list_variacion = np.zeros(N_perm)
    for perm in tqdm(range(N_perm)):
        sampleo_controles = np.random.choice(variacion_controles_diff_condicion, 20)
        data_variacion = np.hstack((sampleo_controles, variacion_diff))
        np.random.shuffle(data_variacion)
        d_variacion = np.nanmean(data_variacion[0:20]) - np.nanmean(data_variacion[20:40])
        d_list_variacion[perm] = d_variacion

    x_variacion = np.nanmean(variacion_controles_diff) - np.nanmean(variacion_diff)

    p_variacion = np.sum(d_list_variacion > np.abs(x_variacion))/len(d_list_variacion)
    
    return p_variacion, variacion, variacion_controles

In [ ]:
p_variacion = [0, 0, 0]
p_variacion[0], _, _ = bootstrap_variacion("trans")
p_variacion[1], _, _ = bootstrap_variacion("alter")
p_variacion[2], variacion, variacion_controles = bootstrap_variacion("recup")

df_variacion = pd.DataFrame({"condicion": ["trans", "alter", "recup"], "variacion": p_variacion})

In [ ]:
h, pcorregido = fdrcorrection(df_variacion["variacion"].values, alpha=0.05, method="indep", is_sorted=False)
df_variacion["corregido"] = pcorregido
df_variacion

#### Bootstrap para desvio

In [ ]:
def bootstrap_desvio(condicion: str) -> (float, float):
    condiciones = {"trans": 1, "alter": 2, "recup": 3}
    index = condiciones[condicion]

    connectivity_regions[:,:,np.eye(300).astype(bool)] = np.nan
    connectivity_regions_controles[:,:,np.eye(300).astype(bool)] = np.nan
    connectivity_regions_controles[REMOVE_REPOSO, 0] = np.nan
    connectivity_regions_controles[REMOVE_ALTERACION, 2] = np.nan
    connectivity_regions_controles[REMOVE_REPOSO, 0] = np.nan
    connectivity_regions_controles[REMOVE_ALTERACION, 2] = np.nan
    variacion = np.zeros((20,4))
    desvio = np.zeros((20,4))
    variacion_controles = np.zeros((10,4))
    desvio_controles = np.zeros((10,4))
    for iSess in range(20):
        for iCond in range(4):
            variacion[iSess,iCond] = np.nanstd(connectivity_regions[iSess, iCond, 12:, 12:])/np.nanmean(connectivity_regions[iSess, iCond, 12:, 12:])
            desvio[iSess,iCond] = np.nanstd(connectivity_regions[iSess, iCond, 12:, 12:])

    for iSess in range(10):
        for iCond in range(4):
            variacion_controles[iSess,iCond] = np.nanstd(connectivity_regions_controles[iSess, iCond, 12:, 12:])/np.nanmean(connectivity_regions_controles[iSess, iCond, 12:, 12:])
            desvio_controles[iSess,iCond] = np.nanstd(connectivity_regions_controles[iSess, iCond, 12:, 12:])

    variacion_diff = variacion[:,index] - variacion[:,0]
    variacion_controles_diff = variacion_controles[:,index] - variacion_controles[:,0]
    variacion_controles_diff_condicion = variacion_controles_diff[~np.isnan(variacion_controles_diff)]
    desvio_diff = desvio[:,index] - desvio[:,0]
    desvio_controles_diff = desvio_controles[:,index] - desvio_controles[:,0]
    desvio_controles_diff_condicion = desvio_controles_diff[~np.isnan(desvio_controles_diff)]

    N_perm = 50_000
    d_list_variacion = np.zeros(N_perm)
    for perm in tqdm(range(N_perm)):
        sampleo_controles = np.random.choice(desvio_controles_diff_condicion, 20)
        data_variacion = np.hstack((sampleo_controles, desvio_diff))
        np.random.shuffle(data_variacion)
        d_variacion = np.nanmean(data_variacion[0:20]) - np.nanmean(data_variacion[20:40])
        d_list_variacion[perm] = d_variacion

    x_variacion = np.nanmean(desvio_controles_diff) - np.nanmean(desvio_diff)

    p_variacion = np.sum(d_list_variacion > np.abs(x_variacion))/len(d_list_variacion)
    
    return p_variacion, variacion, variacion_controles, desvio, desvio_controles

In [ ]:
p_variacion = [0, 0, 0]
p_variacion[0], _, _, _, _ = bootstrap_desvio("trans")
p_variacion[1], _, _, _, _ = bootstrap_desvio("alter")
p_variacion[2], variacion, variacion_controles, desvio, desvio_controles = bootstrap_desvio("recup")

df_variacion = pd.DataFrame({"condicion": ["trans", "alter", "recup"], "variacion": p_variacion})

In [ ]:
h, pcorregido = fdrcorrection(df_variacion["variacion"].values, alpha=0.05, method="indep", is_sorted=False)
df_variacion["corregido"] = pcorregido
df_variacion

#### Bootstrap para inter e intra

In [ ]:
def bootstrap_inter_intra(condicion: str) -> (float, float):
    condiciones = {"trans": 1, "alter": 2, "recup": 3}
    index = condiciones[condicion]

    intra_diff_avp = intra_sessions[:,index]-intra_sessions[:,0]
    inter_diff_avp = inter_sessions[:,index]-inter_sessions[:,0]
    intra_diff_controles = intra_sessions_controles[:,index]-intra_sessions_controles[:,0]
    inter_diff_controles = inter_sessions_controles[:,index]-inter_sessions_controles[:,0]
    inter_diff_controles = inter_diff_controles[~np.isnan(inter_diff_controles)]
    intra_diff_controles = intra_diff_controles[~np.isnan(intra_diff_controles)]

    N_perm = 50_000
    d_list_intra = np.zeros(N_perm)
    d_list_inter = np.zeros(N_perm)
    for perm in tqdm(range(N_perm)):

        data_intra = np.hstack((np.random.choice(intra_diff_controles, 20), intra_diff_avp))
        data_inter = np.hstack((np.random.choice(inter_diff_controles, 20), inter_diff_avp))
        
        np.random.shuffle(data_intra)
        np.random.shuffle(data_inter)

        d_intra = np.nanmean(data_intra[0:20]) - np.nanmean(data_intra[20:40])
        d_inter = np.nanmean(data_inter[0:20]) - np.nanmean(data_inter[20:40])

        d_list_intra[perm] = d_intra
        d_list_inter[perm] = d_inter

    x_intra = np.nanmean(intra_diff_controles) - np.nanmean(intra_diff_avp)
    x_inter = np.nanmean(inter_diff_controles) - np.nanmean(inter_diff_avp)

    p_intra = np.sum(d_list_intra > np.abs(x_intra))/len(d_list_intra)
    p_inter = np.sum(d_list_inter > np.abs(x_inter))/len(d_list_inter)
    
    return p_intra, p_inter, d_list_intra, d_list_inter, x_intra, x_inter

In [ ]:
p_intra = [0, 0, 0]
p_inter = [0, 0, 0]
p_intra[0], p_inter[0], d_list_intra_trans, d_list_inter_trans, x_intra_trans, x_inter_trans = bootstrap_inter_intra("trans")
p_intra[1], p_inter[1], d_list_intra_alter, d_list_inter_alter, x_intra_alter, x_inter_alter = bootstrap_inter_intra("alter")
p_intra[2], p_inter[2], d_list_intra_recup, d_list_inter_recup, x_intra_recup, x_inter_recup = bootstrap_inter_intra("recup")

df_inter_intra = pd.DataFrame({"condicion": ["trans", "alter", "recup"], "intra": p_intra, "inter": p_inter})

In [ ]:
h, pcorregido = fdrcorrection(df_inter_intra["intra"].values, alpha=0.05, method="indep", is_sorted=False)
df_inter_intra["intra_corregido"] = pcorregido

h, pcorregido = fdrcorrection(df_inter_intra["inter"].values, alpha=0.05, method="indep", is_sorted=False)
df_inter_intra["inter_corregido"] = pcorregido
df_inter_intra

#### Figura intra AVP

In [ ]:
plot_prob_points(
    intra_sessions[:,0], 
    intra_sessions[:,1], 
    intra_sessions[:,2], 
    intra_sessions[:,3], 
    facecolors="b", 
    alpha=0.25,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    figsize=(14,10)
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"{MAIN_DIR}/scripts/figuras-finales/conectividad-global/figuras/avp_intra.png")

#### Figura inter AVP

In [ ]:
plot_prob_points(
    inter_sessions[:,0], 
    inter_sessions[:,1], 
    inter_sessions[:,2], 
    inter_sessions[:,3], 
    facecolors=["b","b","b","b"], 
    alpha=[0.25, 0.25, 0.25, 0.25],
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    figsize=(14,10)
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"{MAIN_DIR}/scripts/figuras-finales/conectividad-global/figuras/avp_inter.png")

#### Figura bootstrap VIS SMD

In [ ]:
x_boostrap, x = hist_transicion["VIS_SMD"]
plot_histogram(x_boostrap, x)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"{MAIN_DIR}/scripts/figuras-finales/conectividad-global/figuras/avp_boostrap.png")

#### Figura bootstrap inter intra

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(10,7))
plt.rcParams.update({'font.size': 20})

ax[0][0].hist(d_list_inter_trans, int(np.sqrt(50_000)), density=True)
ax[0][0].plot([x_inter_trans, x_inter_trans], [0, 10], c="red")
ax[0][0].set_title("inter trans")

ax[0][1].hist(d_list_inter_alter, int(np.sqrt(50_000)), density=True)
ax[0][1].plot([x_inter_alter, x_inter_alter], [0, 10], c="red")
ax[0][1].set_title("inter alter")

ax[0][2].hist(d_list_inter_recup, int(np.sqrt(50_000)), density=True)
ax[0][2].plot([x_inter_recup, x_inter_recup], [0, 10], c="red")
ax[0][2].set_title("inter recup")

ax[1][0].hist(d_list_intra_trans, int(np.sqrt(50_000)), density=True)
ax[1][0].plot([x_intra_trans, x_intra_trans], [0, 10], c="red")
ax[1][0].set_title("intra trans")

ax[1][1].hist(d_list_intra_alter, int(np.sqrt(50_000)), density=True)
ax[1][1].plot([x_intra_alter, x_intra_alter], [0, 10], c="red")
ax[1][1].set_title("intra alter")

ax[1][2].hist(d_list_intra_recup, int(np.sqrt(50_000)), density=True)
ax[1][2].plot([x_intra_recup, x_intra_recup], [0, 10], c="red")
ax[1][2].set_title("intra recup")

plt.tight_layout()
fig.show() 

#### Variacion std/mean

In [ ]:
plot_prob_points(
    variacion[:,0], 
    variacion[:,1], 
    variacion[:,2], 
    variacion[:,3], 
    facecolors=["b","b","b","b"], 
    alpha=[0.25, 0.25, 0.25, 0.25],
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    figsize=(14,10)
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"{MAIN_DIR}/scripts/figuras-finales/conectividad-global/figuras/avp_variacion.png")

#### Plano inter-intra

In [ ]:
colors_networks = np.array([
    "#FF0000",  # Red DMN
    "#00FF00",  # Green VIS
    "#0000FF",  # Blue FP
    "#FFFF00",  # Yellow Reward
    "#FF00FF",  # Magenta DA
    "#00FFFF",  # Cyan VA
    "#800080",  # Purple SN
    "#FFA500",  # Orange CO
    "#008000",  # Dark Green SM Dorsal
    "#800000",  # Maroon SM Lateral
    "#008080",  # Teal AUD
    "#000080",  # Navy ParietoMedial
    "#FFD700",  # Gold Medial Temporal Lobe
])

desde = 0
hasta = 1
fig, ax = plt.subplots(figsize=(14,10))
ax.scatter(
    intra_per_network[desde,:], 
    inter_per_network[desde,:], 
    s=100,
    color=colors_networks,
    edgecolor="black"
)
#plt.scatter(
#    intra_per_network[hasta,:], 
#    inter_per_network[hasta,:], 
#    s=100,
#    color=colors_networks,
#    edgecolor="black"
#)


for iFunc in range(13):
    prop = dict(arrowstyle="-|>,head_width=0.15,head_length=0.20",shrinkA=1,shrinkB=1,lw=10,color=colors_networks[iFunc])

    plt.annotate(
        "", 
        xy=(intra_per_network[hasta,iFunc], inter_per_network[hasta,iFunc]), 
        xytext=(intra_per_network[desde,iFunc], inter_per_network[desde,iFunc]), 
        arrowprops=prop
    )

ax.spines[['right', 'top']].set_visible(False)
plt.setp(ax.spines.values(), linewidth=8)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"{MAIN_DIR}/scripts/figuras-finales/conectividad-global/figuras/avp_flechas_transicion.png")

In [ ]:
reposo_mean_inter = inter_sessions[:, 0].mean()
reposo_mean_intra = intra_sessions[:, 0].mean()

reposo_norm_inter = inter_sessions / reposo_mean_inter
reposo_norm_intra = intra_sessions / reposo_mean_intra

X = np.array([1,2,3,4])

fig, ax = plt.subplots(figsize=(14,10))
ax.bar(X-0.2, reposo_norm_inter.mean(axis=0), 0.4, yerr=reposo_norm_inter.std(axis=0)/np.sqrt(20), color="#333333", capsize=7, error_kw=dict(lw=5, capsize=7, capthick=4))
ax.bar(X+0.2, reposo_norm_intra.mean(axis=0), 0.4, yerr=reposo_norm_intra.std(axis=0)/np.sqrt(20), color="#aaaaaa", capsize=7, error_kw=dict(lw=5, capsize=7, capthick=4))
ax.set_xticks([], "")

ax.spines[['right', 'top']].set_visible(False)
plt.setp(ax.spines.values(), linewidth=8)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.ylim(.45,1.12)
plt.savefig(f"{MAIN_DIR}/scripts/figuras-finales/conectividad-global/figuras/inter_intra_normalizada_barras.png")

In [ ]:
X = np.array([1,2,3,4])

fig, ax = plt.subplots(figsize=(14,10))
ax.bar(X, connectivity[:,:,0,2].mean(axis=0), 0.4, yerr=connectivity[:,:,0,2].std(axis=0)/np.sqrt(20), color="#333333", capsize=7, error_kw=dict(lw=5, capsize=7, capthick=4))
#ax.bar(X+0.2, inter_per_network_3d[:,:,2].mean(axis=0), 0.4, yerr=inter_per_network_3d[:,:,2].std(axis=0)/np.sqrt(20), color="#aaaaaa", capsize=7, error_kw=dict(lw=5, capsize=7, capthick=4))
ax.set_xticks([], "")

ax.spines[['right', 'top']].set_visible(False)
plt.setp(ax.spines.values(), linewidth=8)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
#plt.ylim(.45,1.12)
plt.savefig(f"{MAIN_DIR}/scripts/figuras-finales/conectividad-global/figuras/dmn_fp.png")